In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
import numpy as np

In [2]:
logits = tf.random.uniform(minval=0, maxval=1, seed=42, shape=(5, 6))
logits

<tf.Tensor: shape=(5, 6), dtype=float32, numpy=
array([[0.95227146, 0.67740774, 0.79531825, 0.75578177, 0.4759556 ,
        0.6310148 ],
       [0.18602037, 0.11430776, 0.3362218 , 0.723335  , 0.21919966,
        0.8573376 ],
       [0.82392037, 0.59541273, 0.00296271, 0.24725473, 0.5060741 ,
        0.36158872],
       [0.04483843, 0.97216606, 0.82838845, 0.41458404, 0.601666  ,
        0.33958685],
       [0.5731027 , 0.23625553, 0.67478454, 0.00732017, 0.02091265,
        0.9618796 ]], dtype=float32)>

In [ ]:
Q_marginal = tfd.MixtureSameFamily(
mixture_distribution=tfd.Categorical(logits=tf.zeros(tf.shape(logits)[0])),
    components_distribution=tfd.RelaxedBernoulli(logits=tf.transpose(logits),
                                                 temperature=1e-5)
)
Q_marginal

In [ ]:
x = Q_marginal.sample(512)
x = tf.clip_by_value(x, 1e-7, 1.-1e-7)
tf.reduce_mean(-1. * tf.reduce_sum(Q_marginal.log_prob(x), axis=1))


In [ ]:
tf.reduce_mean(-1. * Q_marginal.log_prob(x))

In [ ]:
tfd.RelaxedBernoulli(logits=logits[0], temperature=1e-5).sample(5)

In [ ]:
tf.reshape(tf.repeat(tf.one_hot(tf.range(5), 5), 4, axis=0), shape=(5, 4, 5))

In [ ]:
logits = tf.random.uniform(shape=(4, 6))

In [ ]:
tf.stack([logits] * 5)

In [ ]:
tf.stack([tf.one_hot(tf.range(5), 5)] * 5)

In [ ]:
tf.reshape(tf.repeat(tf.expand_dims(tf.one_hot(2, 5), axis=0), 3, axis=0), shape=(3, 5))

In [ ]:
tf.stack([tf.one_hot(2, 5)] * 3)

In [ ]:
latent_state_size = 3
batch_size = 6
action_size = 2

action = tf.random.uniform(
    shape=(batch_size, ), minval=0, maxval=action_size, dtype=tf.int32, seed=42)
action = tf.one_hot(action, action_size)

latent_states_logits = []
for _ in range(action_size):
    latent_states_logits.append(
        tf.random.uniform(shape=(batch_size, latent_state_size), seed=42))
# latent_states_stack = tf.stack(latent_states_logits, axis=1)

latent_states_stack = tf.random.uniform(shape=(batch_size, action_size, latent_state_size))
action_expand_dims = tf.expand_dims(action, -1)
action_tile = tf.tile(action_expand_dims, (1, 1, latent_state_size))
mult = action_tile * latent_states_stack
latent_states = tf.reduce_sum(mult, axis=1)

print("action", action)
print("latent_states_stack", latent_states_stack)
print("action_tile", action_tile)
print("mult", mult)
print(latent_states)

In [ ]:
print(tf.tile(action_expand_dims, (1, 1, latent_state_size)))
print(tf.reshape(tf.repeat(action_expand_dims, latent_state_size), shape=(batch_size, action_size, latent_state_size)))
print(tf.repeat(action_expand_dims, latent_state_size, axis=-1))

In [ ]:
layers = []
# layers.append(tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, -1)))
layers.append(tf.keras.layers.RepeatVector(latent_state_size))
layers.append(tf.keras.layers.Permute(dims=(2, 1)))
sequential = tf.keras.Sequential(layers)
sequential(action)

In [ ]:
labels_logits = tf.random.uniform(shape=(8, 3))
latent_states_logits = tf.random.uniform(shape=(8, 7))
joint_distribution = tfd.JointDistributionSequential(
    [
        tfd.Independent(tfd.Bernoulli(logits=labels_logits, dtype=tf.float32)),
        lambda l: tfd.Independent(
            tfd.Logistic(loc=tf.concat([latent_states_logits, l], axis=-1) / 0.5,
                               scale=1. / 0.5))
    ], validate_args=True
)
samples = joint_distribution.sample()
samples

In [ ]:
next_label = tf.round(tf.random.uniform(shape=(8, 3), maxval=1, dtype=tf.float32))
next_latent_state = tf.random.uniform(shape=(8, 10))
next_label, next_latent_state

In [ ]:
joint_distribution.log_prob(samples)

In [ ]:
joint_distribution.log_prob_parts(samples)

In [ ]:
tfd.Independent(tfd.Bernoulli(logits=labels_logits, dtype=tf.float32)).kl_divergence(
    tfd.Independent(tfd.Bernoulli(logits=labels_logits/3., dtype=tf.float32)),
)

In [ ]:
d = tfd.Independent(tfd.Bernoulli(logits=labels_logits, dtype=tf.float32))
d.log_prob(samples[0])

In [ ]:
latent_states_logits = tf.random.uniform(shape=(2, 5))
print(latent_states_logits)
tf.tile(tf.expand_dims(latent_states_logits, axis=0), [2, 1, 1])

In [ ]:
logits = tf.random.uniform(shape=(8, 5, 2))
print(logits)
print(tf.transpose(logits, perm=[0, 2, 1]))

In [ ]:
tfd.Independent(tfd.Bernoulli(logits=tf.transpose(logits, perm=[0, 2, 1])), reinterpreted_batch_ndims=1)

In [ ]:
np.unpackbits(np.array([[1, 3, 5, 256]], np.uint8)).reshape(-1, 16)

In [11]:
x = tf.range(2**12)
latent_state_size = 12
tf.map_fn(lambda n: (n // 2 ** tf.range(latent_state_size)) % 2, x)
# print(tf.reduce_sum(_x * 2**tf.range(12), axis=-1))

<tf.Tensor: shape=(4096, 12), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 1, ..., 1, 1, 1],
       [0, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]], dtype=int32)>

In [2]:
states =      tf.constant([3, 3, 2, 2, 2, 2, 2, 3, 3])
actions =     tf.constant([0, 0, 0, 0, 0, 1, 0, 1, 1])
next_states = tf.constant([1, 1, 2, 1, 2, 4, 2, 5, 3])
print(states, actions, next_states)

tf.Tensor([3 3 2 2 2 2 2 3 3], shape=(9,), dtype=int32) tf.Tensor([0 0 0 0 0 1 0 1 1], shape=(9,), dtype=int32) tf.Tensor([1 1 2 1 2 4 2 5 3], shape=(9,), dtype=int32)


In [3]:
num_states = 6
num_actions = 2

_transitions = (states * num_actions * num_states +  # index of the state group
               actions * num_states +  # index of the action group
               next_states)  # index of the next state
print(_transitions)
transitions, _, count = tf.unique_with_counts(_transitions)

print(transitions)
print(count)

tf.Tensor([37 37 26 25 26 34 26 47 45], shape=(9,), dtype=int32)
tf.Tensor([37 26 25 34 47 45], shape=(6,), dtype=int32)
tf.Tensor([2 3 1 1 1 1], shape=(6,), dtype=int32)


In [4]:
_, idx, _ = tf.unique_with_counts(transitions // num_states)
e, _, state_action_pairs_count = tf.unique_with_counts(_transitions // num_states)
print(e)
print(idx)
print(state_action_pairs_count)

tf.Tensor([6 4 5 7], shape=(4,), dtype=int32)
tf.Tensor([0 1 1 2 3 3], shape=(6,), dtype=int32)
tf.Tensor([2 4 1 2], shape=(4,), dtype=int32)


In [5]:
new_transitions = tf.stack([transitions // (num_states * num_actions),
                            (transitions // num_states) % num_actions,
                            transitions % num_states],
                           axis=-1)
print(states)
print(actions)
print(next_states)
print(new_transitions)
print(count)

tf.Tensor([3 3 2 2 2 2 2 3 3], shape=(9,), dtype=int32)
tf.Tensor([0 0 0 0 0 1 0 1 1], shape=(9,), dtype=int32)
tf.Tensor([1 1 2 1 2 4 2 5 3], shape=(9,), dtype=int32)
tf.Tensor(
[[3 0 1]
 [2 0 2]
 [2 0 1]
 [2 1 4]
 [3 1 5]
 [3 1 3]], shape=(6, 3), dtype=int32)
tf.Tensor([2 3 1 1 1 1], shape=(6,), dtype=int32)


In [6]:
sparse_transitions_count = tf.sparse.SparseTensor(
    indices=tf.cast(new_transitions, dtype=tf.int64),
    values=tf.cast(count, tf.float32),
    dense_shape=[num_states, num_actions, num_states])
sparse_transitions_count = tf.sparse.reorder(sparse_transitions_count)
print(sparse_transitions_count)

SparseTensor(indices=tf.Tensor(
[[2 0 1]
 [2 0 2]
 [2 1 4]
 [3 0 1]
 [3 1 3]
 [3 1 5]], shape=(6, 3), dtype=int64), values=tf.Tensor([1. 3. 1. 2. 1. 1.], shape=(6,), dtype=float32), dense_shape=tf.Tensor([6 2 6], shape=(3,), dtype=int64))


In [7]:
sparse_count = tf.sparse.reduce_sum(sparse_transitions_count, axis=-1, output_is_sparse=True)
print(sparse_count)


SparseTensor(indices=tf.Tensor(
[[2 0]
 [2 1]
 [3 0]
 [3 1]], shape=(4, 2), dtype=int64), values=tf.Tensor([4. 1. 2. 2.], shape=(4,), dtype=float32), dense_shape=tf.Tensor([6 2], shape=(2,), dtype=int64))


In [8]:
tf.print('sparse_count\n', sparse_count.indices)
tf.print('transition_count\n', sparse_transitions_count.indices[..., :-1])

sparse_count
 [[2 0]
 [2 1]
 [3 0]
 [3 1]]
transition_count
 [[2 0]
 [2 0]
 [2 1]
 [3 0]
 [3 1]
 [3 1]]


In [9]:
probs = tf.Variable(tf.cast(sparse_transitions_count.values, dtype=tf.float32), trainable=False)
indices = sparse_transitions_count.indices[..., :-1]
i = tf.Variable(0, trainable=False)
j = tf.Variable(0, trainable=False)
while i < tf.shape(probs)[0]:
    tf.print('<i=', i, ', j=', j, '> : ', indices[i], sparse_count.indices[j])
    if tf.reduce_all(indices[i] == sparse_count.indices[j], axis=-1):
        tf.print('  ', indices[i], '==', sparse_count.indices[j])
        tf.print('    ', sparse_transitions_count.values[i], '/', sparse_count.values[j], '=',
              sparse_transitions_count.values[i] / sparse_count.values[j])
        probs[i].assign(sparse_transitions_count.values[i] / sparse_count.values[j])
        i.assign_add(1)
    else:
        j.assign_add(1)  # works only if indices are ordered


<i= 0 , j= 0 > :  [2 0] [2 0]
   [2 0] == [2 0]
     1 / 4 = 0.25
<i= 1 , j= 0 > :  [2 0] [2 0]
   [2 0] == [2 0]
     3 / 4 = 0.75
<i= 2 , j= 0 > :  [2 1] [2 0]
<i= 2 , j= 1 > :  [2 1] [2 1]
   [2 1] == [2 1]
     1 / 1 = 1
<i= 3 , j= 1 > :  [3 0] [2 1]
<i= 3 , j= 2 > :  [3 0] [3 0]
   [3 0] == [3 0]
     2 / 2 = 1
<i= 4 , j= 2 > :  [3 1] [3 0]
<i= 4 , j= 3 > :  [3 1] [3 1]
   [3 1] == [3 1]
     1 / 2 = 0.5
<i= 5 , j= 3 > :  [3 1] [3 1]
   [3 1] == [3 1]
     1 / 2 = 0.5


In [10]:
probs

<tf.Variable 'Variable:0' shape=(6,) dtype=float32, numpy=array([0.25, 0.75, 1.  , 1.  , 0.5 , 0.5 ], dtype=float32)>

In [9]:
probs = tf.reduce_sum(
    tf.map_fn(
        fn=lambda _x: tf.where(
            condition=tf.reduce_all(sparse_transitions_count.indices[..., :-1] == _x[0], axis=-1),
            x=sparse_transitions_count.values / _x[1],
            y=tf.zeros(tf.shape(sparse_transitions_count.values))),
        elems=(sparse_count.indices, sparse_count.values),
        dtype=tf.float32),
    axis=0)

In [10]:
probs

<tf.Tensor: shape=(6,), dtype=float32, numpy=array([0.25, 0.75, 1.  , 1.  , 0.5 , 0.5 ], dtype=float32)>

In [13]:
sparse_transition_probs = tf.sparse.reorder(
    tf.sparse.SparseTensor(
        indices=sparse_transitions_count.indices,
        values=probs,
        dense_shape=sparse_transitions_count.dense_shape))
print(sparse_transition_probs)

SparseTensor(indices=tf.Tensor(
[[2 0 1]
 [2 0 2]
 [2 1 4]
 [3 0 1]
 [3 1 3]
 [3 1 5]], shape=(6, 3), dtype=int64), values=tf.Tensor([0.25 0.75 1.   1.   0.5  0.5 ], shape=(6,), dtype=float32), dense_shape=tf.Tensor([6 2 6], shape=(3,), dtype=int64))


In [61]:
print(tf.sparse.slice(sparse_transition_probs, [2, 0, 0], [1, 1, num_states]))

SparseTensor(indices=tf.Tensor(
[[0 0 1]
 [0 0 2]], shape=(2, 3), dtype=int64), values=tf.Tensor([0.25 0.75], shape=(2,), dtype=float32), dense_shape=tf.Tensor([1 1 6], shape=(3,), dtype=int64))


In [60]:
tf.size(tf.sparse.slice(sparse_transition_probs, [1, 0, 0], [1, 1, num_states]).values)

<tf.Tensor: shape=(), dtype=int32, numpy=0>

In [50]:
print(tf.sparse.slice(sparse_transition_probs, [2, 0, 1], [1, num_actions, num_states]))

SparseTensor(indices=tf.Tensor(
[[0 0 0]
 [0 0 1]
 [0 1 3]], shape=(3, 3), dtype=int64), values=tf.Tensor([0.25 0.75 1.  ], shape=(3,), dtype=float32), dense_shape=tf.Tensor([1 2 5], shape=(3,), dtype=int64))


In [41]:
print(tf.sparse.reduce_sum(sparse_transition_probs, axis=[1, 2], output_is_sparse=True))

SparseTensor(indices=tf.Tensor(
[[2]
 [3]], shape=(2, 1), dtype=int64), values=tf.Tensor([2. 2.], shape=(2,), dtype=float32), dense_shape=tf.Tensor([6], shape=(1,), dtype=int64))


In [43]:
tf.squeeze(tf.sparse.reduce_sum(sparse_transition_probs, axis=[1, 2], output_is_sparse=True).indices)

<tf.Tensor: shape=(2,), dtype=int64, numpy=array([2, 3])>

In [63]:
tf.squeeze(tf.sparse.slice(sparse_transition_probs, [3, 1, 1], [1, 1, 1]).values)

<tf.Tensor: shape=(0,), dtype=float32, numpy=array([], dtype=float32)>

In [81]:
tf.equal(tf.size(tf.squeeze(tf.sparse.slice(sparse_transition_probs, [2, 0, 1], [1, 1, 1]).values)), 0)

<tf.Tensor: shape=(), dtype=bool, numpy=False>

In [160]:
tf.equal(tf.size(tf.squeeze(tf.sparse.slice(sparse_transition_probs, [3, 1, 1], [1, 1, 1]).values)), 0)

<tf.Tensor: shape=(), dtype=bool, numpy=True>

In [37]:
tf.print(tf.cast(sparse_transitions_count, tf.bool))

'SparseTensor(indices=[[2 0 1]
 [2 0 2]
 [2 1 4]
 [3 0 1]
 [3 1 3]
 [3 1 5]], values=[1 1 1 1 1 1], shape=[6 2 6])'


In [13]:
enabled_actions = tf.cast(tf.sparse.reduce_sum(sparse_transition_probs, axis=-1, output_is_sparse=True), tf.bool)

In [14]:
enabled_actions.dtype

tf.bool

In [15]:
tf.squeeze(tf.sparse.slice(enabled_actions, [2, 0], [1, 1]).values)

<tf.Tensor: shape=(), dtype=bool, numpy=True>

In [16]:
tf.squeeze(tf.sparse.slice(enabled_actions, [3, 3], [1, 1]).values)

<tf.Tensor: shape=(0,), dtype=bool, numpy=array([], dtype=bool)>

In [19]:
@tf.function
def is_ok():
    if tf.squeeze(tf.sparse.slice(enabled_actions, [2, 0], [1, 1]).values):
        tf.print('[2, 0] ok')
    if tf.squeeze(tf.sparse.slice(enabled_actions, [3, 3], [1, 1]).values):
        tf.print('[3, 3] ok')

In [20]:
is_ok()

[2, 0] ok


InvalidArgumentError:  Input to reshape is a tensor with 0 values, but the requested shape has 1
	 [[node Reshape_1 (defined at <ipython-input-19-cb8f75230ebe>:5) ]] [Op:__inference_is_ok_223]

Function call stack:
is_ok


In [34]:
tf.reduce_all(tf.squeeze(tf.sparse.to_dense(tf.sparse.slice(enabled_actions, [2, 0], [1, 1]))))

<tf.Tensor: shape=(), dtype=bool, numpy=True>

In [35]:
tf.reduce_all(tf.squeeze(tf.sparse.to_dense(tf.sparse.slice(enabled_actions, [3, 3], [1, 1]))))

<tf.Tensor: shape=(), dtype=bool, numpy=True>

In [39]:
tf.reduce_all([])

<tf.Tensor: shape=(), dtype=bool, numpy=True>

In [41]:
tf.size([])

<tf.Tensor: shape=(), dtype=int32, numpy=0>